In [134]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout

In [135]:
def get_ttnews():
    url = 'https://www.ttnews.com/articles'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        titleText = title.getText()

        if (publish_date != None and titleText != None and text != None):
            companies.append('TTNews')
            description.append(text)
            publish_date.append(date)
            stories.append(title.get('href'))
            titles.append(titleText)
            images.append(image)
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

In [136]:
ttnews = get_ttnews()

In [137]:
def get_freightWaves():
    url = 'https://www.freightwaves.com/news/category/trucking'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text)
        publish_date.append(date)
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        

In [138]:
fw_news = get_freightWaves()

In [139]:
final = pd.concat([ttnews, fw_news], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [140]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Senate Passes Funding Bill to Avert Government...,[ The Senate passed a stopgap spending bill Oc...,"[September 30, 2021]",/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
1,TTNews,Transport Topics Radio,[ Transport Topics Radio is a weekly news roun...,"[September 30, 2021]",/transport-topics-radio,https://ttnews.com/,[]
2,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"[ AUSTIN, Texas — The leader of one of the nat...","[September 30, 2021]",/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
3,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,[ A medical advisory board to the Federal Moto...,"[September 30, 2021]",/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
4,TTNews,Lordstown Nears Deal to Sell Ohio Plant to Tai...,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]
5,TTNews,"Lawmakers, Stakeholders Press for Port Infrast...",[ WASHINGTON — Funding included in a $3.5 tril...,"[September 30, 2021]",/articles/lawmakers-stakeholders-press-port-in...,https://ttnews.com/,[]
6,TTNews,New EV Charger Can Provide 62 Miles of Range i...,[ ABB Ltd. is preparing to roll out the first ...,"[September 30, 2021]",/articles/new-ev-charger-can-provide-62-miles-...,https://ttnews.com/,[]
7,TTNews,Car Sales Drop as Chip Shortages Stymie Buyers,[ After sounding an optimistic note in the fir...,"[September 30, 2021]",/articles/car-sales-drop-chip-shortages-stymie...,https://ttnews.com/,[]
8,TTNews,"Capitol Cliffhanger: Federal Funding, Infrastr...",[ WASHINGTON — His government overhaul plans a...,"[September 29, 2021]",/articles/progress-budget-talks-will-deal-beat...,https://ttnews.com/,[]
9,TTNews,What Q4 and Beyond Have in Store for Transport...,[ The outlook for transportation and logistics...,"[September 29, 2021]",/articles/what-q4-and-beyond-have-store-transp...,https://ttnews.com/,[]


In [141]:
final.head()

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Senate Passes Funding Bill to Avert Government...,[ The Senate passed a stopgap spending bill Oc...,"[September 30, 2021]",/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
1,TTNews,Transport Topics Radio,[ Transport Topics Radio is a weekly news roun...,"[September 30, 2021]",/transport-topics-radio,https://ttnews.com/,[]
2,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"[ AUSTIN, Texas — The leader of one of the nat...","[September 30, 2021]",/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
3,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,[ A medical advisory board to the Federal Moto...,"[September 30, 2021]",/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
4,TTNews,Lordstown Nears Deal to Sell Ohio Plant to Tai...,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]


In [142]:
len(final)

28

In [143]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [144]:
len(final)

28

In [145]:
final = final[final.company != 'Bolt']

In [146]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Senate Passes Funding Bill to Avert Government...,[ The Senate passed a stopgap spending bill Oc...,"[September 30, 2021]",/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
1,TTNews,Transport Topics Radio,[ Transport Topics Radio is a weekly news roun...,"[September 30, 2021]",/transport-topics-radio,https://ttnews.com/,[]
2,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"[ AUSTIN, Texas — The leader of one of the nat...","[September 30, 2021]",/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
3,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,[ A medical advisory board to the Federal Moto...,"[September 30, 2021]",/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
4,TTNews,Lordstown Nears Deal to Sell Ohio Plant to Tai...,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]
5,TTNews,"Lawmakers, Stakeholders Press for Port Infrast...",[ WASHINGTON — Funding included in a $3.5 tril...,"[September 30, 2021]",/articles/lawmakers-stakeholders-press-port-in...,https://ttnews.com/,[]
6,TTNews,New EV Charger Can Provide 62 Miles of Range i...,[ ABB Ltd. is preparing to roll out the first ...,"[September 30, 2021]",/articles/new-ev-charger-can-provide-62-miles-...,https://ttnews.com/,[]
7,TTNews,Car Sales Drop as Chip Shortages Stymie Buyers,[ After sounding an optimistic note in the fir...,"[September 30, 2021]",/articles/car-sales-drop-chip-shortages-stymie...,https://ttnews.com/,[]
8,TTNews,"Capitol Cliffhanger: Federal Funding, Infrastr...",[ WASHINGTON — His government overhaul plans a...,"[September 29, 2021]",/articles/progress-budget-talks-will-deal-beat...,https://ttnews.com/,[]
9,TTNews,What Q4 and Beyond Have in Store for Transport...,[ The outlook for transportation and logistics...,"[September 29, 2021]",/articles/what-q4-and-beyond-have-store-transp...,https://ttnews.com/,[]


In [147]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

In [148]:
len(new_old)

28

In [149]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [150]:
print(title)
pyperclip.copy(title)

<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>Sunday, 08/29/2021</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>


In [151]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        {img}
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [152]:
pyperclip.copy(content)
print(content)


<tr>
      <td>
        <img alt="Capitol Hill" height="293" src="https://h7f7z2r7.stackpathcdn.com/sites/default/files/styles/image-528-293/public/images/articles/senate-main.jpg?itok=YA8-52bS" typeof="foaf:Image" width="528"/>
        <h3>
          <a href = "https://ttnews.com//articles/senate-passes-funding-bill-avert-government-shutdown">Senate Passes Funding Bill to Avert Government Shutdown</a>
        </h3>
        <p>
          <p> The Senate passed a stopgap spending bill Oct. 1 aimed at averting a federal government shutdown on a bipartisan 65-35 vote. The House is expected to pass the measure and send it to President Joe Biden for signature before the midnight deadline.</p>
        </p>
        <p>
          <a href = "https://ttnews.com/">TTNews</a>
        </p>
        <p>
          <b><span class="date-display-single" content="2021-09-30T15:00:00-04:00" datatype="xsd:dateTime" property="dc:date">September 30, 2021</span></b>
        </p>
      </td>
    </tr>
<tr>
    

In [153]:
#job boards
# stackoverflow, jobvite, bamboohr